# Il verme è tratto
Cercherò di creare una trasposizione di questo gioco da tavola, se mai ci riesca.

## Regolamento
Documento contenente le regole

In [26]:
%%HTML
<iframe src=./regole.pdf width=100% height=350></iframe>

## Analisi
Note sul problema

### Svolgimento
In questa sezione cercherò di spiegare i passaggi della partita per comprendere meglio l'algoritmo da adottare

<span class="mark">1. Si decide il numero di giocatori (da 2 a 7) e li si ordina per età dal più giovane al più vecchio (in origine si gioca in senso orario a partire dal più giovane)</span>
<span class="mark">2. Si dispongono le tessere sul tavolo a formare una griglia in ordine numerico (da 21 a 36)</span>
3. Il primo giocatore lancia i dadi:
    1. lancia 8 dadi e sceglie quale serie tenere con lo stesso numero in comune e può rilanciare i dadi rimanenti fino a quando:
        * ottiene un numero maggiore o uguale a una tessera disponibie dalla griglia del tavolo
        * ottiene un numero uguale ad una tessera in cima ad una pila di un altro giocatore
        * non ci sono più serie con numeri in comune non precedentemente trattenuti (non si possono scegliere piu volte serie con lo stesso numero in comune)
        * non si è almeno trattenuto una volta una serie con il simbolo del verme
        * non ci sono più dadi da rilanciare
    3. il giocatore prende una una tessera e la mette in cima alla sua pila se:
        * ha totalizzato un numero maggiore o uguale ad una delle tessere disposte sulla griglia, prende una di queste
        * ha totalizzato un numero uguale a una delle tessere in cima alla pila di un altro giocatore, prende questa
    4. in tutti gli altri casi ha sballato e quindi, se possiede delle tessere, dovrà:
        * restituire quella in cima alla sua pila reinserendola nella griglia
        * se nella griglia ci sono ancora tessere dal valore più alto di quella restituita bisognerà capovolgere (e quindi rendere indisponibile) la tessera dal valore più alto presente nella griglia
4. Si passa al giocatore seguente
5. Il gioco ha fine quando non ci sono più tessere sottraibili dalla griglia
6. Vince il giocatore che ha totalizzato più vermi (ogni tessera oltre al valore numerico ha anche un punteggio rappresentato da dei vermi, da uno a quattro)

### Idee
Quello che potrei implementare

* Per iniziare potrei gestire il gioco fra persone
* Poi potrei integrare un pochetto di grafica (guizero?)
* Poi potrei inserire intelligenza artificiale (difficile, basata sulle probabilità)
* Infine potrei tentare di integrare il gioco in rete

### Architettura
Struttura del programma

Considerando le idee ecco come dovrei strutturare il programma:
1. L'interfaccia è la parte di programma che ci permette di interagire (I/O) con esso e può cambiare
2. I giocatori possono essere sia umani e artificiali, per cui le azioni che gli verranno richieste saranno esplicate manualmente (attraveso l'interfaccia) oppure automaticamente (intelligenza artificiale)
3. Sarà inoltre possibile giocare in rete, questo vuol dire che ci dovranno essere giocatori remoti (struttura client server ?)

Luoghi identificati
- main: colui che gestisce l'esecuzione generale
- interfaccia: tutti gli i/o
- tavolo: tutto quello che è disposto sul tavolo (dadi, tessere, giocatori)
- giocatori: possono essere umani/artificiali, hanno caratteristiche (nome, eta, immagine, indirizzo) e compiono azioni (sequenze dadi, pesca)
- dadi: hanno delle facce e immagini, possono essere lanciati
- tessere: hanno valori e punti e immagini

Funzionamento
Il main avvia l'interfaccia, il tavolo e la partita
L'inter

### Grafico
rappresentazione struttura

## Codice

In [6]:
import doctest, random, itertools, os

In [28]:
class Interfaccia:
    """
    Questa classe deve contenere le azioni eseguite in base al tipo di interfaccia implementata
    """
    
    def inizializza_giocatori(self):
        """metodo che richiede i giocatori e ritorna una loro lista"""
        raise NotImplementedError

    def scelta_dadi(self):
        """metodo che gestisce i lanci di un giocatore"""
        raise NotImplementedError

In [ ]:
class Tui(Interfaccia):
    def inizializza_giocatori(self):
        """richiede partecipanti, restituisce lista di giocatori"""
        #numero giocatori
        gioca_num = int(input("inserisci numero giocatori(2-7): "))
        gioca_num = min(max(2, gioca_num), 7)   #no minore di 2, non maggioree di 7
        nomi = []
        giocatori = []
        #input loop nome e età 
        while len(giocatori) < gioca_num:
            nome = input("inserisci nome giocatore: ")
            if nome in nomi:
                print("Giocatore già esistente, usare un altro nome")
                continue
            nomi.append(nome)
            while True:
                eta  = input("inserisci la sua età o niente per random: ")
                try:
                    eta = int(eta)
                except ValueError:
                    if eta == "":                     #se non si specifica l'età
                        eta = random.randint(0,100)   #viene estratta random
                        print("Età random: {} anni".format(eta))
                finally:
                    if isinstance(eta, int): break
                    else: print("età sbagliata, ritenta")
            giocatori.append(G_Umano(nome, eta, self))
        #restituisco i giocatori
        return giocatori
   
    def formatta(self, lista):
        """restituisce una stringa con il contenuto di una lista. la lista viene ordinata,
        gli elementi ripetuti vengono accoppiati uno affianco all'altro senza separatori,
        i gruppi di elementi diversi vengono separati da uno spazio, senza virgolette
        """
        stringa = ' '.join(str(el)*lista.count(el) for el in sorted(set(lista)))
        return stringa

    def scelta_dadi(self, tavolo):
        dadi_disp = list(tavolo.dadi)
        scelti = []
        totale = 0
        while True:
            pescati = [dado.lancia() for dado in dadi_disp]
            print("totale:{}, scelti:{}, pescati:{}".format(totale, self.formatta(scelti), self.formatta(pescati)))

                  
def lanci_serie(lanci=8):
    """esegue la procedura di estrazione dei numero per n lanci e restituisce il totale delle estrazioni ottenute
    oppure None se si è sballato
    potrebbe essere un metodo del giocatore subclassato umano
    """
    pescati = []
    scelti = []
    totale = 0
    giocabile = continua = True
    while continua and any([comb not in scelti for comb in Dado._dado]):  #nei num. scelti non sono presenti tutti i numeri di un dado
        pescati = Dado.lancia(lanci)
        print("totale:{}, scelti:{}, pescati:{}".format(totale, formatta(scelti), formatta(pescati)))
        giocabile = any([comb not in scelti for comb in pescati])   #nei num. scelti non sono presenti tutti i numeri delle pescate
        if not giocabile:
            print("non puoi più scegliere niente, hai perso questa mano")
            break
        while True:
            scelto = input("scegli un valore fra quelli pescati o [s]top: ")
            if scelto == 's':
                continua = False
                break
            comparse = pescati.count(scelto)
            if comparse and scelto not in scelti:
                scelti.append(scelto)
                totale += int(scelto)*comparse if scelto != 'v' else 5*comparse
                lanci -= comparse
                break
            else:
                print('scelta non valida')
    if giocabile:
        return totale
    else:
        return None

In [ ]:
class Gui(Interfaccia):
    """
        PER ORA LA LASCIO COSÌ MA LE IMMAGINI È MEGLIO GENERARLE COME DICEVA WOLF
        Esempio di come potrebbe funzionare gui, ad esempio ho reimplementato i metodi
        per le tessere e i dadi così da aggiungergli le immagini
        
        Aggiunge le immagini ai dadi
        >>> g = Gui()
        >>> d = g.inizializza_dadi()
        >>> fd = ('dado_gif/1.gif', 'dado_gif/2.gif', 'dado_gif/3.gif', 'dado_gif/4.gif', 'dado_gif/5.gif', 'dado_gif/6.gif')
        >>> all(dado.facced == fd for dado in d)
        True
        >>> from os.path import isfile
        >>> all([isfile(dado) for dado in fd])
        True
        
        Riesce ad utilizzare il metodo della classe padre per generare la lista tessere
        >>> g = Gui()
        >>> t = g.inizializza_tessere()
        >>> t
        [Tessera(21, 1), Tessera(22, 1), Tessera(23, 1), Tessera(24, 1), Tessera(25, 2), Tessera(26, 2), Tessera(27, 2), Tessera(28, 2), Tessera(29, 3), Tessera(30, 3), Tessera(31, 3), Tessera(32, 3), Tessera(33, 4), Tessera(34, 4), Tessera(35, 4), Tessera(36, 4)]
    
        E inoltre gli aggiunge dei path validi per le immagini
        >>> from os.path import isfile
        >>> all([isfile(tes.immagine) for tes in t])
        True
    """
    def inizializza_dadi(self):
        dadi = super().inizializza_dadi()
        facced = tuple(os.path.join("dado_gif/", str(f) + ".gif") for f in dadi[0].faccev)
        for dado in dadi:
            dado.facced = facced
        return dadi
            
    
    def inizializza_tessere(self):
        tessere = super().inizializza_tessere()
        for tessera in tessere:
            tessera.immagine = os.path.join("tessere_gif/", str(tessera.valore) + ".gif")
        return tessere

In [3]:
class Giocatore():
    """
    Questa classe rappresenta un giocatore
    
    Istanziando il giocatore bisogna specificare il nome e l'età (facoltativa)
    >>> g = Giocatore('aldo', 14)
    >>> g.nome  #il nome del giocatore
    'aldo'
    >>> g.eta   #la sua età
    14
    
    Senza età viene assegnata un età casuale da 0 a 100
    >>> g = Giocatore('rino', 22)
    >>> isinstance(g.eta, int) and 0 <= g.eta <= 100
    True
    

    """
    def __init__(self, nome, eta):
        self.nome = nome
        self.eta = eta
  
    def __repr__(self):
        # return f'{type(self).__name__}("{self.nome}", {self.eta})'
        return '{}("{}", {})'.format(type(self).__name__, self.nome, self.eta)

In [31]:
class G_Umano(Giocatore):
    """se il giocatore è umano, i/o sarà gestita da una interfaccia
    
    >>> g = G_Umano('aldo', '35')
    >>> isinstance(g.ui, Interfaccia), g.eta, g.nome
    (True, 35, 'aldo')
    >>> g
    G_Umano("aldo", 35)
    
    Età non valida: ValueError
    >>> g = G_Umano('rinco', 'asda')
    Traceback (most recent call last):
    ...
    ValueError: invalid literal for int() with base 10: 'asda'
    
    """
    def __init__(self, nome, eta=None, ui=Interfaccia()):
        if eta is None: eta = random.randint(0,100) #età casuale
        else: eta = int(eta)
        super().__init__(nome, eta)
        self.ui = ui   #l'interfaccia serve per giocatori diversi da umani locali, a questo punto generalizzo

    def scelta_dadi(self, tavolo):
        risultato = self.ui.scelta_dadi(tavolo)
        

In [43]:
class G_Artificiale(Giocatore):
    """se il giocatore è un ia, le azioni saranno calcolate in automatico
    attualmente quanto fatto è inutile perché ci sono molte più variabili
    da tenere in considerazione (il fatto che i vermi valgono 5 ma bisogna
    considerarli a parte per le estrazioni, il fatto che le probabilità
    devono essere calcolate per le coppie dei lanci, il fatto che non
    è più possibile ripescare numeri già usciti...), insomma per ora
    si salverebbe solo combinazioni ma modificandolo e tenendo solo
    le combinazioni dei lanci e non le somme
    
    >>> ia = G_Artificiale('asd', 17)
    >>> c = ia.combinazioni(3)
    >>> len(c) == 6**3
    True
    
    >>> p = ia.possibilità(c, 4)
    >>> print(p)
    1.3888888888888888
    
    >>> tot = 0
    >>> for r in set(c): tot += ia.possibilità(c, r)
    >>> print(round(tot, 6), "%", sep='')
    100.0%
    
    >>> ia.perc_riuscita(3, 4)
    1.3888888888888888
    
    >>> ia.perc_riuscita(2, 10)
    11.11111111111111
    
    >>> sorted(ia._perc_riuscita.keys())
    [2, 3]
    """
    _perc_riuscita = dict()
    
    def combinazioni(self, ndadi=1, facce=6):
        """restituisce i risultati delle somme di tutte
        le combinazioni possibili dei lanci dei dadi
        ho trasformato i 6 in 5 (ma non è ancora una soluzione ottimale)
        """
        if facce == 0: return []
        comb = [(1,)*ndadi] #[(1,1,1)]
        risu = [ndadi]      #[sum(comb[0])]
        maxl = facce**ndadi #6**3
        cont = 0
        while cont < maxl:
            cm = list(comb[cont])
            cont += 1
            for i in range(ndadi):
                if cm[i] < facce:
                    cm[i] += 1
                    comb.append(tuple(cm))
                    # somma = sum(cm)
                    # traformo i 6 in 5 per le somme
                    somma = sum(map(lambda x: min(5, x), cm))
                    risu.append(somma)
                    break
                else:
                    cm[i] = 1
        #~ return comb, sorted(risu), maxl
        return sorted(risu)

    def possibilità(self, combinazioni, estrazione):
        """restituisce la percentuale che esca un valore
        fra delle combinazioni
        """
        return 100/len(combinazioni)*combinazioni.count(estrazione)

    def perc_riuscita(self, numero_dadi, risultato):
        """restituisce la possibilità in percentuale che esca un determinato
        numero da un lancio di dadi, i lanci vengono effettuati solo la prima
        volta e poi i risultati memorizzati in un dizionario
        """
        percentuali = self._perc_riuscita.get(numero_dadi)
        if percentuali is not None:
            return percentuali.get(risultato, 0)
        else:
            self._perc_riuscita.update({numero_dadi : dict()})
            risultati = self.combinazioni(numero_dadi)
            for ris_univ in set(risultati):
                percentuale = self.possibilità(risultati, ris_univ)
                self._perc_riuscita[numero_dadi].update({ris_univ : percentuale})
            return self._perc_riuscita[numero_dadi].get(risultato, 0)

doctest.testmod()

TestResults(failed=0, attempted=16)

In [33]:
class G_Remoto(Giocatore):
    """se il giocatore è remoto l'io sarà gestito da un client, il client dovrà vedere che interfaccia usare"""
    pass

In [34]:
class Tessera():
    """
    Questa classe rappresenta una tessera
    
    >>> t = Tessera(21, 1)
    >>> t
    Tessera(21, 1)
    """
    def __init__(self, valore, punti):
        self.valore = int(valore)
        self.punti = int(punti)
    
    def __repr__(self):
        return "Tessera({}, {})".format(self.valore, self.punti)

In [35]:
class Dado():
    """
    Questa classe rappresenta un dado
    
    >>> d = Dado(6)
    >>> d.faccev
    (1, 2, 3, 4, 5, 6)
    
    >>> d.lancia() in [1,2,3,4,5,6]
    True
    """
    def __init__(self, facce=6):
        self.faccev = tuple(range(1, facce+1))
    
    def __repr__(self):
        return "Dado({})".format(len(self.faccev))
        
    # def lancia(self, numero=1):
    #     """Lancia il dado n volte"""
    #     return [random.choice(self.faccev) for n in range(numero)]  
    
    def lancia(self):
        """lancia il dado una sola volta"""
        return random.choice(self.faccev)

In [36]:
class Tavolo():
    """
    Questa classe serve a gestire gli spostamenti sul tavolo
    
    >>> 
    
    disattivo questo test perché è interattivo, comunuqe inserendo come input
    2, a, 2, b, 3, funziona correttamente
    >> tui = Tui()
    >> tav = Tavolo(tui)
    >> isinstance(tav.giocatori[0], Giocatore)
    True
    >> isinstance(tav.griglia[0], Tessera)
    True
    >> isinstance(tav.dadi[0], Dado)
    True
    """
    def __init__(self, giocatori):
        self.giocatori = self.ordina_giocatori(giocatori)
        self.griglia = self.inizializza_griglia_tessere()
        self.dadi = self.inizializza_dadi()
        # ui.tavolo = self    #dipendenza circolare!
    
    def inizializza_griglia_tessere(self):
        """Questa funzione genera una lista di tessere"""
        tessere = []
        p = 1
        for v in range (21, 37):
            tessere.append(Tessera(v, int(p)))
            p += 0.25
        return tessere
    
    def inizializza_dadi(self):
        """Crea una tupla con tutti i dadi in gioco"""
        return tuple(Dado() for i in range(8))
    
    def ordina_giocatori(self, giocatori):
        """ordina i giocaatori per età"""
        # g = giocatori.sort(key=lambda giocatore:giocatore.eta) #sbagliato
        return sorted(giocatori, key=lambda giocatore:giocatore.eta) 

    def turno(self):
        """restituisce il giocatore del turno corrente fintanto che ci sono tessere nella griglia"""
        print(type(self.giocatori), self.giocatori)
        giocatore = itertools.cycle(self.giocatori)
        while self.griglia:
            yield next(giocatore)
        else:  #fine tessere
            return None

In [ ]:
### programma
if __name__ == "__main__":
    doctest.testmod()                             #eseguo i test
    ui = Tui()                                    #scelgo l'interfaccia
    tv = Tavolo(ui.inizializza_giocatori())       #inizializzo il tavolo

    #provo a fare un giro di gioco, poi vedrò come e dove piazzarlo
    for giocatore in tv.turno():
        risultato = giocatore.scelta_dadi(tv)        
        # if a == 'x': break
    
    
    

inserisci numero giocatori(2-7): 1
inserisci nome giocatore: a
inserisci la sua età o niente per random: 
Età random: 91 anni
inserisci nome giocatore: b
inserisci la sua età o niente per random: 
Età random: 12 anni
<class 'list'> [G_Umano("b", 12), G_Umano("a", 91)]
[2, 6, 3, 6, 2, 2, 6, 1]


### codice inutilizzato

In [ ]:
def inizializza_griglia_tessere(car="tessere_gif/", est=".gif"):
    """
    Questa funzione genera una lista di tessere
    
    >>> ts = genera_tessere()
    >>> ts
    [(21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 2), (27, 2), (28, 2), (29, 3), (30, 3), (31, 3), (32, 3), (33, 4), (34, 4), (35, 4), (36, 4)]
    >>> from os.path import isfile
    >>> all([isfile(t.immagine) for t in ts])
    True
    """
    tessere = []
    p = 1
    for v in range (21, 37):
        path = car + str(v) + est
        tessere.append(Tessera(v, int(p), path))
        p += 0.25
    return tessere

In [ ]:
#metto i doctest dentro un altra funzione alla fine, se non li voglio nelle docstring
#di ogni singola classe e funzione potrei fare così e metterli a parte
def test_giocatore():
    """
    Questa classe rappresenta un giocatore
    
    Istanziando il giocatore bisogna specificare il nome e l'età (facoltativa)
    >>> g = Giocatore('aldo', 14)
    >>> g.nome  #il nome del giocatore
    'aldo'
    >>> g.eta   #la sua età
    14
    
    Senza età viene assegnata un età casuale da 0 a 100
    >>> g = Giocatore('rino')
    >>> isinstance(g.eta, int) and 0 <= g.eta <= 100
    True
    
    Età non valida sempre età casuale
    >>> g = Giocatore('rinco', 'asda')
    >>> isinstance(g.eta, int) and 0 <= g.eta <= 100
    True
    
    Età valida
    >>> g = Giocatore('rinco', '134')
    >>> g.eta
    134
    """
    pass
doctest.testmod()

In [17]:
!jupyter nbconvert --to script verme_interfacce2.ipynb

[NbConvertApp] Converting notebook verme_interfacce2.ipynb to script
[NbConvertApp] Writing 36622 bytes to verme_interfacce2.py
